# Reference Forecasts

This [Jupyter notebook](https://jupyter.org) is designed to introduce you to the [Solar Forecast Arbiter's](https://solarforecastarbiter.org/) built-in reference forecast capabilities. It is divided into 2 sections:

1. [NWP-based forecasts](#NWP-based-forecasts)
2. [Persistence forecasts](#persistence-forecasts)

The API documentation is available [here](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html).

In [1]:
import datetime
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd

from bokeh.core.properties import value
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Category10_10 as PALETTE
TOOLS = "pan,box_zoom,xwheel_zoom,reset,save"
output_notebook()

Loading BokehJS ...

In [2]:
from solarforecastarbiter import datamodel

## NWP-based forecasts

Forecasts based on NWP model data are used for intraday and longer forecasts. The Solar Forecast Arbiter contains a set of functions to process data from NWP forecasts. Here, we explore some of the functionality with an emphasis on obtaining results. See the [NWP section of the documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html#nwp) for additional information.

The `solarforecastarbiter` package includes a handful of subsetted NWP model runs for testing. We'll use these for our demonstration below. The models were all initialized at 2019-05-15 00Z and the subsets include about half a degree of latitude and longitude near Tucson, AZ.

In [3]:
from solarforecastarbiter.io import nwp
# find the files
base_path = Path(nwp.__file__).resolve().parents[0] / 'tests/data'
# define file loading function that knows where to find the files
load_forecast = partial(nwp.load_forecast, base_path=base_path)

In [4]:
# define coordinates
latitude = 32.2
longitude = -110.9
elevation = 700

# define initialization time, forecast start time, forecast end time
init_time = pd.Timestamp('20190515T0000Z')
start = pd.Timestamp('20190515T0100Z')
end = pd.Timestamp('20190518T0000Z')

The Solar Forecast Arbiter organizes its metadata and data according to the ideas described in the [data model](https://solarforecastarbiter.org/datamodel/). The `solarforecastarbiter` package implements this data model in `solarforecastarbiter.datamodel`. Here, we define a `Site` object for which to make forecasts.

In [5]:
# define a Site object
site = datamodel.Site(
    name='Tucson, AZ',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix'
)

Next we select a NWP+post processing model, load and process the data, and plot the processed data.

Here we use the subhourly irradiance directly from the HRRR model.

In [6]:
# import the relevant python modules
from solarforecastarbiter.reference_forecasts import main, models

# select the model
model = models.hrrr_subhourly_to_subhourly_instantaneous

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. ac_power is None because site is not a power plant.
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

# plotting details...
data = dict(zip(('GHI', 'DNI', 'DHI', 'Air temperature', 'Wind speed', 'index'),
                (ghi, dni, dhi, air_temperature, wind_speed, ghi.index)))
source = ColumnDataSource(data)

fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=200)
palette = iter(PALETTE)
fig1 = figure(**fig_kwargs)
for name in ('GHI', 'DNI', 'DHI'):
    fig1.line(x='index', y=name, source=source, legend=value(name), color=next(palette), line_width=2)
fig1.yaxis.axis_label = "Irradiance (W/m^2)"
fig1.legend.location = "top_left"

fig_kwargs['x_range'] = fig1.x_range  # link x-zoom
fig2 = figure(**fig_kwargs)
fig2.line(x='index', y='Air temperature', source=source, legend=value("Air temperature"), color=next(palette), line_width=2)
fig2.yaxis.axis_label = "Air temperature (C)"
fig2.legend.location = "top_left"

fig3 = figure(**fig_kwargs)
fig3.line(x='index', y='Wind speed', source=source, legend=value("Wind speed"), color=next(palette), line_width=2)
fig3.yaxis.axis_label = "Wind speed (m/s)"
fig3.legend.location = "top_left"

grid = gridplot([fig1, fig2, fig3], ncols=1, plot_width=800)

show(grid)

The Solar Forecast Arbiter includes processing functions derive GHI from a NWP model's cloud cover forecast and estimate the DNI and DHI using the [Erbs model](https://pvlib-python.readthedocs.io/en/stable/generated/pvlib.irradiance.erbs.html?highlight=erbs). These functions are applied to data from the GFS, NAM, and RAP weather models. 

For example, GFS data is processed as follows:

1. Load time series forecast of cloud cover, air temperature, and wind speed from gridded data.
2. Convert mixed interval cloud cover in raw data into standard intervals.
2. Interpolate cloud cover, air temperature, wind speed to 5 minute resolution to minimize errors due to solar position.
2. Compute GHI from cloud cover.
2. Compute DNI and DHI from GHI.
2. If desired, calculate AC power.
2. Resample 5 minute data to 1 hour data.

See the `models` [documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.models.html#module-solarforecastarbiter.reference_forecasts.models) for more explanation.

In [7]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. ac_power is None because site is not a power plant.
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

# plotting details...
data = dict(zip(('GHI', 'DNI', 'DHI', 'Air temperature', 'Wind speed', 'index'),
                (ghi, dni, dhi, air_temperature, wind_speed, ghi.index)))
source = ColumnDataSource(data)

fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=200)
palette = iter(PALETTE)
fig1 = figure(**fig_kwargs)
for name in ('GHI', 'DNI', 'DHI'):
    fig1.line(x='index', y=name, source=source, legend=value(name), color=next(palette), line_width=2)
fig1.yaxis.axis_label = "Irradiance (W/m^2)"
fig1.legend.location = "top_left"

fig_kwargs['x_range'] = fig1.x_range  # link x-zoom
fig2 = figure(**fig_kwargs)
fig2.line(x='index', y='Air temperature', source=source, legend=value("Air temperature"), color=next(palette), line_width=2)
fig2.yaxis.axis_label = "Air temperature (C)"
fig2.legend.location = "top_left"

fig3 = figure(**fig_kwargs)
fig3.line(x='index', y='Wind speed', source=source, legend=value("Wind speed"), color=next(palette), line_width=2)
fig3.yaxis.axis_label = "Wind speed (m/s)"
fig3.legend.location = "top_left"

grid = gridplot([fig1, fig2, fig3], ncols=1, plot_width=800)

show(grid)

The forecast function will produce an AC power forecast if supplied with the appropriate metadata. Here we define a single axis tracker.

In [8]:
# define the modeling parameters
modeling_parameters = datamodel.SingleAxisModelingParameters(
    ac_capacity=10,  # always in MW
    dc_capacity=13,
    temperature_coefficient=-0.003,  # 1/C
    dc_loss_factor=0,
    ac_loss_factor=0,
    axis_tilt=0,
    axis_azimuth=0,
    ground_coverage_ratio=0.4,
    backtrack=True,
    max_rotation_angle=50,
)
# define a SolarPowerPlant object
plant = datamodel.SolarPowerPlant(
    name='Tucson AZ Plant',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix',
    modeling_parameters=modeling_parameters
)

Now we run the same code as before to create a forecast from the GFS model.

In [9]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data. only difference from before is plant replaces site
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(plant, model_wrapped, init_time, start, end)

# plotting details...
data = dict(zip(('GHI', 'DNI', 'DHI', 'Air temperature', 'Wind speed', 'AC power', 'index'),
                (ghi, dni, dhi, air_temperature, wind_speed, ac_power, ac_power.index)))
source = ColumnDataSource(data)

fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=200)
palette = iter(PALETTE)
fig1 = figure(**fig_kwargs)
for name in ('GHI', 'DNI', 'DHI'):
    fig1.line(x='index', y=name, source=source, legend=value(name), color=next(palette), line_width=2)
fig1.yaxis.axis_label = "Irradiance (W/m^2)"
fig1.legend.location = "top_left"

fig_kwargs['x_range'] = fig1.x_range  # link x-zoom
fig2 = figure(**fig_kwargs)
fig2.line(x='index', y='Air temperature', source=source, legend=value("Air temperature"), color=next(palette), line_width=2)
fig2.yaxis.axis_label = "Air temperature (C)"
fig2.legend.location = "top_left"

fig3 = figure(**fig_kwargs)
fig3.line(x='index', y='Wind speed', source=source, legend=value("Wind speed"), color=next(palette), line_width=2)
fig3.yaxis.axis_label = "Wind speed (m/s)"
fig3.legend.location = "top_left"

fig4 = figure(**fig_kwargs)
fig4.line(x='index', y='AC power', source=source, legend=value("AC power"), color=next(palette), line_width=2)
fig4.yaxis.axis_label = "AC power (MW)"
fig4.xaxis.axis_label = 'Time (UTC)'
fig4.legend.location = "top_left"

grid = gridplot([fig1, fig2, fig3, fig4], ncols=1, plot_width=800)

show(grid)

## Persistence forecasts

The solarforecastarbiter supports several varieties of persistence forecasts:

1. Persistence of observed values
2. Persistence of irradiance or power accounting for solar position

The [run_persistence](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.main.run_persistence.html#solarforecastarbiter.reference_forecasts.main.run_persistence) function provides a high-level interface for creating persistence forecasts. Lower level functions with more flexibility may be found in the [persistence](https://solarforecastarbiter-core.readthedocs.io/en/latest/generated/solarforecastarbiter.reference_forecasts.persistence.html#module-solarforecastarbiter.reference_forecasts.persistence) module. This exercise will focus on the high-level interface (see [this gist](https://gist.github.com/wholmgren/3e983c8e465e17e316038ac5a9574c52) for examples using the low level interface).

In [10]:
observation_file_path = 'https://raw.githubusercontent.com/SolarArbiter/2019-Denver-Workshop/master/observation_1h.csv'
observation_values = pd.read_csv(observation_file_path, header=1, index_col=0, parse_dates=True)
observation_values

,value,quality_flag
timestamp,,
2019-06-06 00:00:00+00:00,5,0
2019-06-06 01:00:00+00:00,0,0
2019-06-06 02:00:00+00:00,0,0
2019-06-06 03:00:00+00:00,0,0
2019-06-06 04:00:00+00:00,0,0
2019-06-06 05:00:00+00:00,10,0
2019-06-06 06:00:00+00:00,100,0
2019-06-06 07:00:00+00:00,200,0
2019-06-06 08:00:00+00:00,350,0


In [11]:
observation = datamodel.Observation(
    name='sample observation', 
    interval_length=pd.Timedelta('1hr'),
    interval_label='ending',
    interval_value_type='mean',
    variable='ghi',
    uncertainty=0,
    site=site
)

In [12]:
# change default load_data function so we can use without API access
def load_data_base(data, observation, data_start, data_end):
    # slice doesn't care about closed or interval label 
    # so here we manually adjust start and end times
    if 'instant' in observation.interval_label:
        pass
    elif observation.interval_label == 'ending':
        data_start += pd.Timedelta('1s')
    elif observation.interval_label == 'beginning':
        data_end -= pd.Timedelta('1s')
    return data[data_start:data_end]

In [13]:
from solarforecastarbiter.reference_forecasts import persistence
persistence._load_data = partial(load_data_base, observation_values)

In [14]:
forecast = datamodel.Forecast(
    name='persistence fx', 
    issue_time_of_day=datetime.time(0),  # 0Z
    lead_time_to_start=pd.Timedelta('1hr'),
    interval_length=pd.Timedelta('1hr'),
    run_length=pd.Timedelta('1hr'),
    interval_label='ending',
    interval_value_type='mean',
    variable='ghi',
    site=site
)

In [15]:
issue_time = pd.Timestamp('20190606 1700')
run_time = issue_time
fx_per = main.run_persistence(None, observation, forecast, run_time, issue_time, index=False)
fx_per_idx = main.run_persistence(None, observation, forecast, run_time, issue_time, index=True)

AttributeError: 'NoneType' object has no attribute 'get_observation_values'